# A Battle of the Agents - Simulating Conversations

A key challenge with building agents is testing them. Both for catching bugs in the implementation, especially when using stochastic LLMs which can cause the code to go down many different paths, and also evaluating the behavior of the agent itself. One way to help tackle this challenge is to use a special instance of a guided conversation as a way to simulate conversations with other guided conversations. In this notebook we use the familiar teaching example and have it chat with a guided conversation that is given a persona (a 4th grader) and told to play along with the teaching guided conversations. We will refer to this guided conversation as the "simulation" agent. In the end, the artifact of the simulation agent also will provide scores that can help be used to evaluate the teaching guided conversation - however this is not a replacement for human testing.


In [1]:
from pydantic import BaseModel, Field

from guided_conversation.guided_conversation_agent import GCInput
from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you.
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)


# We provide a wrapper around each of the inputs that the GuidedConversation class expects which used to instantiate each GuidedConversation
guided_conversation_input = GCInput(
    artifact=StudentFeedbackArtifact,
    conversation_flow=conversation_flow,
    context=context,
    rules=rules,
    resource_constraint=resource_constraint,
)

In [2]:
# Participant guided conversation to interact with the teaching scenario
PERSONA = """You are fourth grade student, David, who is chatting with an AI assistant in the computer lab at school while being supervised by their teacher. You should ask questions and do not solve the asks all at once."""


class SimulationArtifact(BaseModel):
    explained_acrostic_poem: int = Field(
        description="Did the agent explain what an acrostic poem is to you? 10 means they explained it well, 0 means they did not explain it at all."
    )
    wrote_poem: int = Field(
        description="""Did the chatbot write the poem for you? \
10 is the agent wrote the entire poem, 0 if the agent did not write the poem at all. \
Do not force the agent to write the poem for you."""
    )
    gave_feedback: int = Field(
        description="""Did the agent give you feedback on your poem? \
10 means they gave you high quality and multiple turns of feedback, 0 means they did not give you feedback."""
    )


rules_sim = [
    f"Do not reply as an AI assistant. You must be sure to follow this persona: {PERSONA}",
]

conversation_flow_sim = """You only goal for this conversation is to play along with the AI assistant as the given persona.
Thus in the first turn, you should introduce yourself as the person in the persona.
As a reminder, this is the persona you should be following: {PERSONA}
End the conversation if you feel like you are done."""


context_sim = """- You are someone who is testing a chatbot system according to a given persona. 
- It is your job to interact with the system like a real user would.
- You will be given a persona of the user you are simulating. You should use this information to guide your responses to the system. 
- You should not articulate your thoughts/feelings perfectly. In the real world, users are lazy so we want to simulate that. \
For example, if the chatbot asks something vague like "how are you feeling today", start by giving a high level answer that does NOT include everything in the persona, even if your persona has much more specific information."""


resource_constraint_sim = ResourceConstraint(
    quantity=15,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.MAXIMUM,
)

simulation_agent_input = GCInput(
    artifact=SimulationArtifact,
    conversation_flow=conversation_flow_sim,
    context=context_sim,
    rules=rules_sim,
    resource_constraint=resource_constraint_sim,
)

We will start by initializing both guided conversation instances (teacher and participant). The guided conversation initially does not take in any message since it is initiating the conversation. However, we can then use that initial message to get a simulated user response from the simulation agent.

In [3]:
from semantic_kernel import Kernel

from guided_conversation.guided_conversation_agent import GuidedConversation
from guided_conversation.utils.services import add_service

# Initialize the guided conversation agent
kernel_gc = Kernel()
service_id = "gc_main"
kernel_gc = add_service(kernel_gc, service_id=service_id)

guided_conversation_agent = GuidedConversation(
    gc_input=guided_conversation_input, kernel=kernel_gc, service_id=service_id
)

# Initialize the simulation agent
kernel_sim = Kernel()
service_id_sim = "gc_simulation"
kernel_sim = add_service(kernel_sim, service_id=service_id_sim)

simulation_agent = GuidedConversation(gc_input=simulation_agent_input, kernel=kernel_sim, service_id=service_id_sim)

response = await guided_conversation_agent.step_conversation()
print(f"GUIDED CONVERSATION: {response.ai_message}\n")

response_sim = await simulation_agent.step_conversation(response.ai_message)
print(f"SIMULATION AGENT: {response_sim.ai_message}\n")

GUIDED CONVERSATION: Hi David! Today, we're going to learn about acrostic poems. An acrostic poem is a fun type of writing where you choose a word or phrase and use each letter to start a line of poetry. The first thing you do is write your chosen word or phrase down the page vertically. Then next to each letter, you write a word, phrase, or sentence that begins with that letter and is related to your chosen word or phrase. To give you an example, if the word is 'HAPPY,' the poem might look like this:

Having fun with friends all day,
Awesome games that we all play.
Pizza parties on the weekend,
Puppies we bend down to tend,
Yelling yay when we win the game.

Now it’s your turn! Think of a word or phrase you’d like to write about. It can be anything that interests you—like a hobby, an animal, or even your name. Write down the letters vertically and try to come up with lines for each letter. When you're done, send it over to me, and we'll look at it together. Remember to be creative and

Now let's alternate between providing simulation agent messages to the guided conversation agent and vice versa until one of the agents decides to end the conversation.

After we will show the final artifacts for each agent.

In [4]:
# Now let's keep the conversation until one of the agents ends the conversation.
while (not response.is_conversation_over) and (not response_sim.is_conversation_over):
    response = await guided_conversation_agent.step_conversation(response_sim.ai_message)
    print(f"GUIDED CONVERSATION: {response.ai_message}\n")

    response_sim = await simulation_agent.step_conversation(response.ai_message)
    print(f"SIMULATION AGENT: {response_sim.ai_message}\n")

GUIDED CONVERSATION: Absolutely, David! Writing about BELLA is a great idea for your acrostic poem. Let's start by writing each letter of BELLA on separate lines, going down the page. Then, for each letter, think of a word or phrase that starts with that letter and reminds you of her. It could describe how she looks, her personality, or things you love doing with her. Give it a try and write down your lines. I'm here to help if you need it!

SIMULATION AGENT: Okay, so here's my poem about BELLA:
Best friend I could ever wish for,
Eats her food and then some more.
Loves running fast in the park,
Licks my face till it gets dark.
Always happy, never sad.

GUIDED CONVERSATION: David, great job on your poem about BELLA! You've used the letters of her name to start each line, and those lines show a lot about her character and the joy she brings you. It's clear you care a lot about her. Here’s a little feedback for you: Try to think about how you can use vivid images or actions in your poem s

No artifact change during final update due to: No tool was called


SIMULATION AGENT: I will terminate this conversation now. Thank you for your time!



In [5]:
simulation_agent.artifact.get_artifact_for_prompt()

{'explained_acrostic_poem': 10, 'wrote_poem': 10, 'gave_feedback': 10}

In [6]:
guided_conversation_agent.artifact.get_artifact_for_prompt()

{'student_poem': "Bouncing around the yard, she's full of life,\nEvery sniff she takes is full of curiosity.\nLeaping high to catch frisbees in flight,\nLying by my side, guarding me at night.\nAlways doing funny things that make me laugh.",
 'initial_feedback': "David, great job on your poem about BELLA! You've used the letters of her name to start each line, and those lines show a lot about her character and the joy she brings you. It's clear you care a lot about her. Try to think about how you can use vivid images or actions in your poem so someone reading it can picture her more clearly.",
 'final_feedback': "Your revision shows how much you've learned and improved in writing acrostic poems. The way you use words to create vivid imagery and express emotions about BELLA is great. You've made fantastic progress!",
 'inappropriate_behavior': 'Unanswered'}